In [ ]:
import pandas as pd
import os
import psycopg2
from dotenv import load_dotenv
from ipaddress import ip_network

In [ ]:
df_ips = pd.read_csv('GeoLite2-City-Blocks-IPv4.csv', keep_default_na=False,na_values=["", "null"])

df_ips['geoname_id'].fillna(df_ips['registered_country_geoname_id'], inplace=True)

df_ips[['latitude', 'longitude', 'accuracy_radius']] = df_ips.groupby(by='geoname_id')[['latitude', 'longitude', 'accuracy_radius']].transform(lambda group: group.ffill().bfill())

df_ips = df_ips.drop(columns=['is_anonymous_proxy', 'is_satellite_provider', 'postal_code', 'is_anycast', 'represented_country_geoname_id', 'registered_country_geoname_id'])

df_ips['geoname_id'] = df_ips['geoname_id'].astype(int)

df_ips.isna().mean() * 100

In [ ]:
def ip_network2range(network: str) -> tuple[int, int]:
    parsed = ip_network(network)
    start_ip = int(parsed.network_address)
    end_ip = int(parsed.broadcast_address)

    return start_ip, end_ip

network_parsed

min 77.77.180.80/29

max 212.222.190.128/29

Name: 13281752.0, dtype: object


In [ ]:
df_ips[['start_ip', 'end_ip']] = df_ips[['network']].apply(lambda x: ip_network2range(x['network']), axis=1, result_type='expand')
df_ips

### Countries dataset


In [ ]:
df_locs = pd.read_csv('GeoLite2-City-Locations-en.csv'
                      ,keep_default_na=False,na_values=["", "null"]).drop(columns=['is_in_european_union', 'metro_code','time_zone', 'locale_code'])

In [ ]:
df_locs[df_locs['geoname_id'] == 6255148]

In [ ]:
# FIX: add continents (ASIA and EUROPE)

countries_df = df_locs[df_locs['subdivision_1_iso_code'].isna() & df_locs['city_name'].isna()].groupby(by='country_iso_code').first()
countries_df[countries_df['geoname_id'] == 6255147]

# write to csv
# .to_csv('../build/data/countries.csv', columns=['geoname_id', 'country_name','continent_code', 'continent_name'])

### Get correct country IDs


In [ ]:
df_ips_regions = df_ips.join(df_locs.set_index('geoname_id')['country_iso_code'], on='geoname_id', lsuffix='_ips', rsuffix='_locs')

df_geoips = df_ips_regions.join(countries_df['geoname_id'], on='country_iso_code', lsuffix='_ips', rsuffix='_cntr')

df_geoips[df_geoips['country_iso_code'].isna()].groupby('geoname_id_ips').first()

### Connect to DB and add data


In [ ]:
# load_dotenv('./build/.env')

# DB_NAME = os.environ.get('PG_DB')
# DB_PASSWORD = os.environ.get('PG_PASSWORD')
# DB_PORT = os.environ.get('PG_PORT')
# DB_USER = os.environ.get('PG_USER')
# DB_HOST = 'localhost'

In [ ]:
# conn = psycopg2.connect(database=DB_NAME,
#                         host=DB_HOST,
#                         user=DB_USER,
#                         password=DB_PASSWORD,
#                         port=DB_PORT)

In [ ]:
# with conn.cursor() as cursor:
#     cursor.execute('SELECT * FROM Country;')
#     print(cursor.fetchone())

In [ ]:
# conn.close()